## This file is to prepare the final data for the use in Tableau

In [37]:
import pandas as pd

In [68]:
df_comments_entities = pd.read_parquet('data_after_processing/reviews_related_data/comments_with_entities.parquet')
df_comments_sentiments = pd.read_csv('data_after_processing/reviews_related_data/comments_with_sentiments_imputed.csv')
df_keyphrases = pd.read_parquet('data_after_processing/reviews_related_data/keyphrases_with_index.parquet')

In [69]:
print(df_comments_entities.shape)
print(df_comments_sentiments.shape)
print(df_keyphrases.shape)

(546736, 19)
(546736, 15)
(546736, 5)


In [70]:
df_comments_final = pd.concat([
    df_comments_entities[['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments','clean_comments', 'keyphrases','cleaned_comments', 'expanded_clean_comments', 'sentences', 'entities']],
    df_keyphrases.set_index('index')[['sentences_person', 'sentences_location', 'keyphrases_person','keyphrases_location']],
    df_comments_sentiments[['roberta_neg', 'roberta_pos', 'roberta_neu']]
], axis=1)

In [72]:
df_comments_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546736 entries, 0 to 546735
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   listing_id               546736 non-null  float64
 1   id                       546736 non-null  float64
 2   date                     546736 non-null  object 
 3   reviewer_id              546736 non-null  int64  
 4   reviewer_name            546735 non-null  object 
 5   comments                 546736 non-null  object 
 6   clean_comments           546736 non-null  object 
 7   keyphrases               546736 non-null  object 
 8   cleaned_comments         546736 non-null  object 
 9   expanded_clean_comments  546736 non-null  object 
 10  sentences                546736 non-null  object 
 11  entities                 546736 non-null  object 
 12  sentences_person         546736 non-null  object 
 13  sentences_location       546736 non-null  object 
 14  keyp

In [73]:
# Changing float 64 to int64
df_comments_final['listing_id'] = df_comments_final['listing_id'].astype('int64')
df_comments_final['id'] = df_comments_final['id'].astype('int64')

In [81]:
# Postfix the listing_id that were converted due to type conversions
# This try to figure out the real listing ID from the original data based on reviewer name, comments and date
df_original = pd.read_csv('./data_after_processing/reviews_related_data/processed_reviews.csv', parse_dates=['date'])
df_original = df_original[df_original['listing_id'] > 54409066].copy()
df_original['hash'] = df_original.apply(lambda x: hash(''.join([x['reviewer_name'], x['comments'], x['date'].strftime('%Y-%m-%d')])), axis=1)

for index, row in df_comments_final[df_comments_final['listing_id'] > 54409066].iterrows():
    h = hash(''.join([row['reviewer_name'], row['comments'], row['date']]))
    original = df_original[df_original['hash'] == h]
    if original.shape[0] > 0:
        df_comments_final.loc[index, 'listing_id'] = original.iloc[0].listing_id
        df_comments_final.loc[index, 'id'] = original.iloc[0].id

df_comments_final.listing_id.max()

709952316127237292

In [82]:
df_comments_final.to_csv('data_after_processing/reviews_related_data/comments_final.csv', index=False)

In [83]:
df = df_comments_final.copy()
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546736 entries, 0 to 546735
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   listing_id               546736 non-null  int64  
 1   id                       546736 non-null  int64  
 2   date                     546736 non-null  object 
 3   reviewer_id              546736 non-null  int64  
 4   reviewer_name            546735 non-null  object 
 5   comments                 546736 non-null  object 
 6   clean_comments           546736 non-null  object 
 7   keyphrases               546736 non-null  object 
 8   cleaned_comments         546736 non-null  object 
 9   expanded_clean_comments  546736 non-null  object 
 10  sentences                546736 non-null  object 
 11  entities                 546736 non-null  object 
 12  sentences_person         546736 non-null  object 
 13  sentences_location       546736 non-null  object 
 14  keyp

In [85]:
df[df.isnull().any(axis=1)]


,listing_id,id,date,reviewer_id,reviewer_name,comments,clean_comments,keyphrases,cleaned_comments,expanded_clean_comments,sentences,entities,sentences_person,sentences_location,keyphrases_person,keyphrases_location,roberta_neg,roberta_pos,roberta_neu
154364,3710914,115989408,2016-11-27,24562860,None,this house is amazing. My family really love t...,thi house be amaze my family really love thi c...,"[[""thi house"", 0.583], [""nice house"", 0.5484],...",this house is amazing. my family really love t...,this house is amazing. my family really love t...,"[{'end': 22, 'sentence': 'this house is amazin...",[],,,[],[],0.001727,0.990759,0.007514


# Grouped keyphrases data by listing

* Keyphrases with frequency

In [121]:
# Impute empty person/location keyphrase with overall keyphrase
from ast import literal_eval
import numpy as np

def impute_person(row):
    if row['keyphrases_person'].shape[0] == 0:
        return np.array([x[0] for x in literal_eval(row['keyphrases'])])
    return row['keyphrases_person']

def impute_location(row):
    if row['keyphrases_location'].shape[0] == 0:
        return np.array([x[0] for x in literal_eval(row['keyphrases'])])
    return row['keyphrases_location']

df['keyphrases_person'] = df.apply(impute_person, axis=1)
df['keyphrases_location'] = df.apply(impute_location, axis=1)

In [123]:
def list_concat(s):
    a = s.explode().dropna().reset_index(drop=True).tolist()
    return a

df_listing_keyphrases = df.groupby('listing_id').agg({
    'keyphrases_person': list_concat,
    'keyphrases_location': list_concat
})

In [124]:
# Generate CSV with keyphrase frequency
df_listing_person_keyphrase = df_listing_keyphrases[['keyphrases_person']].explode(column='keyphrases_person').reset_index().value_counts().to_frame('counts')
df_listing_person_keyphrase.to_csv('data_after_processing/reviews_related_data/listings_person_keyphrase.csv')

In [125]:
# Generate CSV with keyphrase frequency
df_listing_location_keyphrase = df_listing_keyphrases[['keyphrases_location']].explode(column='keyphrases_location').reset_index().value_counts().to_frame('counts')
df_listing_location_keyphrase.to_csv('data_after_processing/reviews_related_data/listings_location_keyphrase.csv')